# Final Project
## Step1: Process data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import os

In [2]:
csv = "data/Bank-Telemarketing.csv"
experiment_name="telemarketing_experiment1"

In [3]:
data = pd.read_csv(csv)
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [4]:
data.isna().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

In [5]:
data.job.unique()

array(['housemaid', 'services', 'admin.', 'blue-collar', 'technician',
       'retired', 'management', 'unemployed', 'self-employed', 'unknown',
       'entrepreneur', 'student'], dtype=object)

In [6]:
data.education.unique()

array(['basic.4y', 'high.school', 'basic.6y', 'basic.9y',
       'professional.course', 'unknown', 'university.degree',
       'illiterate'], dtype=object)

In [7]:
data.default.unique()

array(['no', 'unknown', 'yes'], dtype=object)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

### Preprocessing:
Looking over the data there are no NA Values by the looks of things but there are a number of Object values that I'll have to convert into numerical data. I'll assign the classification data into a list so that I can translate future data in api deployment.

In [9]:
X = data.drop("y",axis=1).values
y = data.y.values


In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
le.classes_

array(['no', 'yes'], dtype=object)

In [11]:
# This data will go into model_config later so that I can translate data submited through the api
classmap = []
for i in range(1,10):
    X[:,i] = le.fit_transform(X[:,i])
    print(list(le.classes_))
    classmap.append(list(le.classes_))
X[:,14] = le.fit_transform(X[:,14])
print(le.classes_)
classmap.append(list(le.classes_))

['admin.', 'blue-collar', 'entrepreneur', 'housemaid', 'management', 'retired', 'self-employed', 'services', 'student', 'technician', 'unemployed', 'unknown']
['divorced', 'married', 'single', 'unknown']
['basic.4y', 'basic.6y', 'basic.9y', 'high.school', 'illiterate', 'professional.course', 'university.degree', 'unknown']
['no', 'unknown', 'yes']
['no', 'unknown', 'yes']
['no', 'unknown', 'yes']
['cellular', 'telephone']
['apr', 'aug', 'dec', 'jul', 'jun', 'mar', 'may', 'nov', 'oct', 'sep']
['fri', 'mon', 'thu', 'tue', 'wed']
['failure' 'nonexistent' 'success']


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

## Step2: Create a predictive model and store it in ML Tracking system

### Connecting to ML Tracking api:
This will allow me to track the models I create in the Azure api.
I needed to customize the config file and add "tenant_id" as my system was using old login data.

In [14]:
import mlflow
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
import json

with open("config/config.json") as file:
    config = json.load(file)
interactive_auth=InteractiveLoginAuthentication(tenant_id=config["tenant_id"])
ws = Workspace(subscription_id=config["subscription_id"],
                  resource_group=config["resource_group"],
                  workspace_name=config["workspace_name"],
                  auth=interactive_auth)
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())
mlflow.set_experiment(experiment_name)

### Model Creation:
I create a list of models to test and then loop through and test each one in my model_test function.

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

test_models = [LogisticRegression(random_state = 0),
            KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2),
            SVC(kernel = 'rbf', random_state = 0),
            GaussianNB(),
            DecisionTreeClassifier(criterion="entropy", random_state=0),
            RandomForestClassifier(n_estimators=100, criterion="entropy", random_state=0)]


In [16]:
def model_test(Model):
    model_type = type(Model).__name__
    print(model_type)
    Model.fit(X_train, y_train)

    run_name = "Basic " + model_type + " Experiment"
    print(run_name)
    with mlflow.start_run(run_name=run_name) as run:

        Model.fit(X_train, y_train)
        y_pred = Model.predict(X_test)

        score = accuracy_score(y_test, y_pred)

        # Log model
        mlflow.sklearn.log_model(Model, model_type+"-model")

        # Create metrics
        print(f"score: {score}")

        # Log metrics
        mlflow.log_metric("score", score)

        runID = run.info.run_uuid
        experimentID = run.info.experiment_id

        print(f"Inside MLflow Run with run_id `{runID}` and experiment_id `{experimentID}`")

In [17]:
for model in test_models:
    model_test(model)

LogisticRegression
Basic LogisticRegression Experiment
score: 0.9075018208302986
Inside MLflow Run with run_id `73425a02-ee95-4157-8bce-346a5ba0a134` and experiment_id `160a1463-2b81-4d13-8ef1-1338ea40dd8b`
KNeighborsClassifier
Basic KNeighborsClassifier Experiment
score: 0.9036173828599174
Inside MLflow Run with run_id `13786e04-aa81-476f-b6da-248f7c265f30` and experiment_id `160a1463-2b81-4d13-8ef1-1338ea40dd8b`
SVC
Basic SVC Experiment
score: 0.8971837824714737
Inside MLflow Run with run_id `52a4082e-f0d4-4976-bbf9-17ec7a263567` and experiment_id `160a1463-2b81-4d13-8ef1-1338ea40dd8b`
GaussianNB
Basic GaussianNB Experiment
score: 0.8512988589463462
Inside MLflow Run with run_id `35c394ba-55e5-4a89-822e-c67e98e48adf` and experiment_id `160a1463-2b81-4d13-8ef1-1338ea40dd8b`
DecisionTreeClassifier
Basic DecisionTreeClassifier Experiment
score: 0.8883224083515416
Inside MLflow Run with run_id `a07c9da5-a231-438d-9d3e-f059f6ce0d70` and experiment_id `160a1463-2b81-4d13-8ef1-1338ea40dd8b`

## Step 3: Download Best Performing model and Deploy model as API

In [18]:
# So this is just creating the deployments folder that will house my experiments and data.

try:
    os.mkdir(f"deployments/{experiment_name}")
    os.mkdir(f"deployments/{experiment_name}/api")
    print(f"deployments/{experiment_name} created")

except FileExistsError:
    print(f"deployments/{experiment_name} exists")

except FileNotFoundError:
    os.mkdir("deployments")
    os.mkdir(f"deployments/{experiment_name}")
    os.mkdir(f"deployments/{experiment_name}/api")
    print(f"deployments/{experiment_name} created")

deployments/telemarketing_experiment1 exists


### Model Selection:
So this selects the best performing model from all the run in my experiment and selects the best performing one. Once selected it Download the files associated with the file

In [19]:
exp = ws.experiments[experiment_name]
runs = list(exp.get_runs())
high_score = [0, "run"]
for run in runs:
    try:
        print(run.get_metrics("score"))
        tmp = run.get_metrics("score")["score"]
        if tmp > high_score[0]:
            high_score = [tmp, run]
    except KeyError:
        pass

print(high_score[0],"\n",high_score[1])

{'score': 0.9145423646516144}
{'score': 0.8883224083515416}
{'score': 0.8512988589463462}
{'score': 0.8971837824714737}
{'score': 0.9036173828599174}
{'score': 0.9075018208302986}
{'score': 0.9145423646516144}
{'score': 0.8883224083515416}
{'score': 0.8512988589463462}
{'score': 0.8971837824714737}
{'score': 0.9036173828599174}
{'score': 0.9075018208302986}
{'score': 0.9145423646516144}
{'score': 0.8883224083515416}
{'score': 0.8512988589463462}
{'score': 0.8971837824714737}
{'score': 0.9036173828599174}
{'score': 0.9075018208302986}
{'score': 0.9145423646516144}
{'score': 0.8883224083515416}
{'score': 0.8512988589463462}
{'score': 0.8971837824714737}
{'score': 0.9036173828599174}
{'score': 0.9075018208302986}
{'score': 0.9145423646516144}
{'score': 0.8883224083515416}
{'score': 0.8512988589463462}
{'score': 0.8971837824714737}
{'score': 0.9036173828599174}
{'score': 0.9075018208302986}
{}
{'score': 0.9036173828599174}
{'score': 0.9079873755765963}
{'score': 0.9145423646516144}
{'score

In [20]:
high_score[1].download_files(output_directory=f"deployments/{experiment_name}")
filenames = high_score[1].get_file_names()
filenames

['RandomForestClassifier-model/MLmodel',
 'RandomForestClassifier-model/conda.yaml',
 'RandomForestClassifier-model/model.pkl']

In [21]:
# Runs to find pkl filename and directory its in
for file in filenames:
    if ".pkl" in file:
        file = file.split(r"/")
        dir_name = "/".join(file[:-1])

print(dir_name)

RandomForestClassifier-model


In [22]:
# This is where I create the model_config file for my api to use
# It needs to know the directory structure and how the classes are mapped
model_config_json = {"experiment_name":experiment_name,
                     "model_directory":dir_name,
                     "classmap": classmap}

with open(f"deployments/{experiment_name}/api/model_config.json", "w") as f:
    f.write(json.dumps(model_config_json))

In [23]:
# Copies FlaskAPI template file to Deployments api folder. 
from shutil import copyfile
copyfile("template/FlaskAPI.py", f"deployments/{experiment_name}/api/FlaskAPI.py")

'deployments/telemarketing_experiment1/api/FlaskAPI.py'

### Create Dockerfile to build image from

In [24]:
dir = "./"+dir_name
dir2 = "/usr/local/python/" +dir_name
text = """FROM python:3.7.8
MAINTAINER Alex McClead, Northwestern Student
COPY ./api /usr/local/python/api
COPY {} {}
EXPOSE 5010
WORKDIR /usr/local/python/api/
RUN pip install -r requirements.txt
CMD python FlaskAPI.py
""".format(dir, dir2)
print(text)

FROM python:3.7.8
MAINTAINER Alex McClead, Northwestern Student
COPY ./api /usr/local/python/api
COPY ./RandomForestClassifier-model /usr/local/python/RandomForestClassifier-model
EXPOSE 5010
WORKDIR /usr/local/python/api/
RUN pip install -r requirements.txt
CMD python FlaskAPI.py



In [25]:
with open(f"deployments/{experiment_name}/Dockerfile","w") as f:
    f.write(text)

In [26]:
os.chdir(f"deployments/{experiment_name}")

In [27]:
# Write requirements file to api folder
text ="""flask
flasgger
scikit-learn
pandas
pickle5"""

with open("api/requirements.txt","w") as f:
    f.write(text)

## Step 4: Containerize model and API

In [28]:
container_name = experiment_name.lower().replace("_","")

!docker build -t $container_name:latest -f Dockerfile .

[+] Building 0.0s (0/1)                                                         
[+] Building 0.1s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 326B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.7.8            0.0s
[+] Building 0.3s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 326B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for dock

[+] Building 1.5s (5/10)                                                        
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 326B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.7.8            0.8s
 => [auth] library/python:pull token for registry-1.docker.io              0.0s
 => [internal] load build context                                          0.6s
 => => transferring context: 24.68MB                                       0.6s
 => [1/5] FROM docker.io/library/python:3.7.8@sha256:6ca82c8d96a24523a71f  0.0s
[+] Building 1.6s (5/10)                                                        
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 326B  

[+] Building 2.6s (9/10)                                                        
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 326B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.7.8            0.8s
 => [auth] library/python:pull token for registry-1.docker.io              0.0s
 => [internal] load build context                                          0.8s
 => => transferring context: 36.53MB                                       0.8s
 => CACHED [1/5] FROM docker.io/library/python:3.7.8@sha256:6ca82c8d96a24  0.0s
 => [2/5] COPY ./api /usr/local/python/api                                 0.1s
 => [3/5] COPY ./RandomForestClassifier-model /usr/local/python/RandomFor  0.3s
 => [4/5] WORKDIR /usr/local/python/api

[+] Building 3.6s (9/10)                                                        
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 326B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.7.8            0.8s
 => [auth] library/python:pull token for registry-1.docker.io              0.0s
 => [internal] load build context                                          0.8s
 => => transferring context: 36.53MB                                       0.8s
 => CACHED [1/5] FROM docker.io/library/python:3.7.8@sha256:6ca82c8d96a24  0.0s
 => [2/5] COPY ./api /usr/local/python/api                                 0.1s
 => [3/5] COPY ./RandomForestClassifier-model /usr/local/python/RandomFor  0.3s
 => [4/5] WORKDIR /usr/local/python/api

[+] Building 4.6s (9/10)                                                        
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 326B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.7.8            0.8s
 => [auth] library/python:pull token for registry-1.docker.io              0.0s
 => [internal] load build context                                          0.8s
 => => transferring context: 36.53MB                                       0.8s
 => CACHED [1/5] FROM docker.io/library/python:3.7.8@sha256:6ca82c8d96a24  0.0s
 => [2/5] COPY ./api /usr/local/python/api                                 0.1s
 => [3/5] COPY ./RandomForestClassifier-model /usr/local/python/RandomFor  0.3s
 => [4/5] WORKDIR /usr/local/python/api

...

[+] Building 26.9s (9/10)                                                       
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 326B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.7.8            0.8s
 => [auth] library/python:pull token for registry-1.docker.io              0.0s
 => [internal] load build context                                          0.8s
 => => transferring context: 36.53MB                                       0.8s
 => CACHED [1/5] FROM docker.io/library/python:3.7.8@sha256:6ca82c8d96a24  0.0s
 => [2/5] COPY ./api /usr/local/python/api                                 0.1s
 => [3/5] COPY ./RandomForestClassifier-model /usr/local/python/RandomFor  0.3s
 => [4/5] WORKDIR /usr/local/python/api

[+] Building 27.7s (9/10)                                                       
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 326B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.7.8            0.8s
 => [auth] library/python:pull token for registry-1.docker.io              0.0s
 => [internal] load build context                                          0.8s
 => => transferring context: 36.53MB                                       0.8s
 => CACHED [1/5] FROM docker.io/library/python:3.7.8@sha256:6ca82c8d96a24  0.0s
 => [2/5] COPY ./api /usr/local/python/api                                 0.1s
 => [3/5] COPY ./RandomForestClassifier-model /usr/local/python/RandomFor  0.3s
 => [4/5] WORKDIR /usr/local/python/api

[+] Building 28.6s (10/11)                                                      
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 326B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.7.8            0.8s
 => [auth] library/python:pull token for registry-1.docker.io              0.0s
 => [internal] load build context                                          0.8s
 => => transferring context: 36.53MB                                       0.8s
 => CACHED [1/5] FROM docker.io/library/python:3.7.8@sha256:6ca82c8d96a24  0.0s
 => [2/5] COPY ./api /usr/local/python/api                                 0.1s
 => [3/5] COPY ./RandomForestClassifier-model /usr/local/python/RandomFor  0.3s
 => [4/5] WORKDIR /usr/local/python/api

[+] Building 29.5s (10/11)                                                      
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 326B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.7.8            0.8s
 => [auth] library/python:pull token for registry-1.docker.io              0.0s
 => [internal] load build context                                          0.8s
 => => transferring context: 36.53MB                                       0.8s
 => CACHED [1/5] FROM docker.io/library/python:3.7.8@sha256:6ca82c8d96a24  0.0s
 => [2/5] COPY ./api /usr/local/python/api                                 0.1s
 => [3/5] COPY ./RandomForestClassifier-model /usr/local/python/RandomFor  0.3s
 => [4/5] WORKDIR /usr/local/python/api

[+] Building 30.4s (10/11)                                                      
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 326B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.7.8            0.8s
 => [auth] library/python:pull token for registry-1.docker.io              0.0s
 => [internal] load build context                                          0.8s
 => => transferring context: 36.53MB                                       0.8s
 => CACHED [1/5] FROM docker.io/library/python:3.7.8@sha256:6ca82c8d96a24  0.0s
 => [2/5] COPY ./api /usr/local/python/api                                 0.1s
 => [3/5] COPY ./RandomForestClassifier-model /usr/local/python/RandomFor  0.3s
 => [4/5] WORKDIR /usr/local/python/api

In [54]:
!docker run -d --rm -p5010:5010 --name $container_name $container_name:latest

66ec508c8385be51a1d5386507a67175c8e94eac385f7da1af5ab39ef75ae6cc


In [55]:
!docker ps

CONTAINER ID   IMAGE                             COMMAND                  CREATED         STATUS         PORTS                                       NAMES
66ec508c8385   telemarketingexperiment1:latest   "/bin/sh -c 'python …"   7 seconds ago   Up 5 seconds   0.0.0.0:5010->5010/tcp, :::5010->5010/tcp   telemarketingexperiment1


In [31]:
!docker stop $container_name

telemarketingexperiment1


In [42]:
os.chdir("../..")

## Step 5: Output container to Cloud repository

In [43]:
!az login

The default web browser has been opened at https://login.microsoftonline.com/common/oauth2/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.
You have logged in. Now let us find all the subscriptions to which you have access...
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "6c865334-e96c-4ab6-8247-f534795bfc0f",
    "id": "de5b26f3-b6ce-47cd-a291-0004d9ab88bb",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Azure subscription 1",
    "state": "Enabled",
    "tenantId": "6c865334-e96c-4ab6-8247-f534795bfc0f",
    "user": {
      "name": "amccleadfb@gmail.com",
      "type": "user"
    }
  }
]


In [44]:
!az acr login --name finalregistrycis325

Login Succeeded


In [47]:
os.chdir(f"deployments/{experiment_name}")
!az acr build --image $container_name:latest --registry finalregistrycis325 --file Dockerfile .

Packing source code into tar to upload...
Uploading archived source code from '/var/folders/ff/7hrzbb8d5471gv165drbv3sw0000gn/T/build_archive_82e3186b330b4531b880b69abb1e6f93.tar.gz'...
Sending context (5.066 MiB) to registry: finalregistrycis325...
Queued a build with ID: cj3                                  
Waiting for an agent...
2021/06/05 20:41:09 Downloading source code...
2021/06/05 20:41:11 Finished downloading source code
2021/06/05 20:41:11 Using acb_vol_363f6850-1795-4f89-ad25-f740415143b2 as the home volume
2021/06/05 20:41:11 Setting up Docker configuration...
2021/06/05 20:41:12 Successfully set up Docker configuration
2021/06/05 20:41:12 Logging in to registry: finalregistrycis325.azurecr.io
2021/06/05 20:41:13 Successfully logged into finalregistrycis325.azurecr.io
2021/06/05 20:41:13 Executing step ID: build. Timeout(sec): 28800, Working directory: '', Network: ''
2021/06/05 20:41:13 Scanning for dependencies...
2021/06/05 20:41:14 Successfully scanned dependencies
20

f17789b093c5: Layer already exists
e5df62d9b33a: Layer already exists
7a9460d53218: Layer already exists
593694d7053e: Pushed
b2765ac0333a: Layer already exists
0ced13fcf944: Layer already exists
655877d88b67: Pushed

e43ad9995d71: Pushed
latest: digest: sha256:702349ef59379eaa83232f76b5dc11f1ca7ca4435ebcabc6bd91f47be411434a size: 2850
2021/06/05 20:43:00 Successfully pushed image: finalregistrycis325.azurecr.io/telemarketingexperiment1:latest
2021/06/05 20:43:00 Step ID: build marked as successful (elapsed time in seconds: 73.464377)
2021/06/05 20:43:00 Populating digests for step ID: build...
2021/06/05 20:43:01 Successfully populated digests for step ID: build
2021/06/05 20:43:01 Step ID: push marked as successful (elapsed time in seconds: 33.307865)
2021/06/05 20:43:01 The following dependencies were found:
2021/06/05 20:43:01 
- image:
    registry: finalregistrycis325.azurecr.io
    repository: telemarketingexperiment1
    tag: latest
    digest: sha256:702349ef59379eaa83232f76b5

In [48]:
os.chdir("../..")

In [49]:
# Shows that Container now in registry
!az acr repository list --name finalregistrycis325

[
  "bca-api",
  "telemarketingexperiment1"
]


## Step 6: Deploy Containerized model and API in Kubernetes Cluster

In [50]:
kuber_text = """apiVersion: apps/v1
kind: Deployment
metadata:
  name: {}-prediction
spec:
  replicas: 1
  selector:
    matchLabels:
      app: {}-prediction
  strategy:
    rollingUpdate:
      maxSurge: 1
      maxUnavailable: 1
  minReadySeconds: 5
  template:
    metadata:
      labels:
        app: {}-prediction
    spec:
      nodeSelector:
        "beta.kubernetes.io/os": linux
      containers:
      - name: {}-prediction
        image: finalregistrycis325.azurecr.io/{}
        ports:
        - containerPort: 5010
        resources:
          requests:
            cpu: 250m
          limits:
            cpu: 500m
---
apiVersion: v1
kind: Service
metadata:
  name: {}-prediction
spec:
  type: LoadBalancer
  ports:
  - port: 5010
  selector:
    app: {}-prediction""".format(container_name,container_name,container_name,container_name,container_name,container_name,container_name)

print(kuber_text)
with open(f"deployments/{experiment_name}/kuber.yaml","w") as file:
    file.write(kuber_text)

apiVersion: apps/v1
kind: Deployment
metadata:
  name: telemarketingexperiment1-prediction
spec:
  replicas: 1
  selector:
    matchLabels:
      app: telemarketingexperiment1-prediction
  strategy:
    rollingUpdate:
      maxSurge: 1
      maxUnavailable: 1
  minReadySeconds: 5
  template:
    metadata:
      labels:
        app: telemarketingexperiment1-prediction
    spec:
      nodeSelector:
        "beta.kubernetes.io/os": linux
      containers:
      - name: telemarketingexperiment1-prediction
        image: finalregistrycis325.azurecr.io/telemarketingexperiment1
        ports:
        - containerPort: 5010
        resources:
          requests:
            cpu: 250m
          limits:
            cpu: 500m
---
apiVersion: v1
kind: Service
metadata:
  name: telemarketingexperiment1-prediction
spec:
  type: LoadBalancer
  ports:
  - port: 5010
  selector:
    app: telemarketingexperiment1-prediction


In [57]:
!az aks get-credentials --resource-group FinalProject --name finalproject

Merged "finalproject" as current context in /Users/alexandermcclead/.kube/config


In [58]:
!kubectl apply -f deployments/$experiment_name/kuber.yaml

deployment.apps/telemarketingexperiment1-prediction configured
service/telemarketingexperiment1-prediction unchanged


In [60]:
# Shows it's supposably up but I cannot connect to it for some reason. :(
prediction = container_name+"-prediction"
!kubectl get service $prediction --watch

NAME                                  TYPE           CLUSTER-IP     EXTERNAL-IP   PORT(S)          AGE
telemarketingexperiment1-prediction   LoadBalancer   10.0.221.136   13.86.7.152   5010:30018/TCP   68m
